In [1]:
import os
from psd_tools import PSDImage
from PIL import Image
import pickle
import cv2
from lib.image_processing import transform
import detect_scene as ds

In [2]:
psd = PSDImage.load('/home/damian/test/001.psd')

In [3]:
backcolor_in = False
frame_in = False
line_drawing_in = False
bottom_painting_in = False

i = 0
for layer in psd.layers:
    if layer.name == 'backcolor':
        backcolor_idx = i
        backcolor_in = True
    if layer.name == 'frame':
        frame_idx = i
        frame_in = True
    if layer.name == 'line_drawing':
        line_drawing_idx = i
        line_drawing_in = True
    if layer.name == 'bottom_painting':
        bottom_painting_idx = i
        bottom_painting_in = True    
    i += 1    

In [ ]:
def check_names_in():
    if backcolor_in == False:
        raise NameError("backcolor doesn't exist")
    if frame_in == False:
        raise NameError("frame doesn't exist")
    if line_drawing_in == False:
        raise NameError("line_drawing doesn't exist")
    if bottom_painting_in == False:
        raise NameError("bottom_painting doesn't exist")

In [ ]:
try:
    check_names_in()
except NameError as e:
    print ("NameError :", e)

In [6]:
psd.layers

[<psd_tools.Layer: 'frame', size=1007x3787, x=404, y=348>,
 <psd_tools.Layer: '레이어 11', size=943x1114, x=521, y=1896>,
 <psd_tools.Layer: '레이어 10', size=892x1484, x=524, y=1885>,
 <psd_tools.Layer: 'line_drawing', size=876x1373, x=537, y=1978>,
 <psd_tools.Layer: '레이어 7', size=799x1208, x=613, y=2163>,
 <psd_tools.Layer: 'bottom_painting', size=833x1331, x=556, y=1976>,
 <psd_tools.Layer: '레이어 9', size=866x1350, x=549, y=1918>,
 <psd_tools.Layer: '레이어 8', size=916x1474, x=527, y=1880>,
 <psd_tools.Layer: '스케치-------------------------------', size=1051x1276, x=376, y=2062>,
 <psd_tools.Layer: '레이어 5', size=1425x15274, x=31, y=-4484>,
 <psd_tools.Layer: 'backcolor', size=1500x6000, x=0, y=0>]

In [7]:
psd.layers[frame_idx]

<psd_tools.Layer: 'frame', size=1007x3787, x=404, y=348>

In [8]:
point = (psd.layers[frame_idx].bbox.x1, psd.layers[frame_idx].bbox.y1)

In [9]:
point

(404, 348)

In [10]:
psd.layers = [psd.layers[frame_idx],psd.layers[backcolor_idx]]

In [11]:
backcolor_frame_img = psd.as_PIL_merged()

In [12]:
backcolor_frame_img = backcolor_frame_img.convert('RGB')

In [13]:
backcolor_frame_img.save('/home/damian/test/bf001.png')

In [14]:
del psd

In [15]:
img = cv2.imread('/home/damian/test/bf001.png')[:,:,::-1]

In [16]:
filtered_scene = ds.detect_scenes(img,'/home/damian/test/bf001_dt.png')

color image
final_scene information! : 
x, y, w, h of each scene : 
[[ 554 1915  857 1389]
 [ 745  858   87  355]
 [ 403  347  102  350]]
/home/damian/test/bf001_dt.png : # of scenes = 3


In [17]:
psd = PSDImage.load('/home/damian/test/001.psd')

In [18]:
backcolor_img = psd.layers[backcolor_idx].as_PIL()

In [19]:
psd.layers = [psd.layers[line_drawing_idx], psd.layers[bottom_painting_idx]]

In [20]:
lineDrawing_bottomPainting_img = psd.as_PIL_merged()

In [21]:
green_backcolor = Image.new('RGBA', backcolor_img.size, (0,255,0))
alpha_composite = Image.alpha_composite(green_backcolor, lineDrawing_bottomPainting_img)

In [39]:
a = alpha_composite.convert('RGB')

In [49]:
a.save('/home/damian/test/gb001.png')

In [50]:
img = cv2.imread('/home/damian/test/gb001.png')

In [51]:
i = 0
for x, y, w, h in filtered_scene:
    i += 1
    center_x = x + int(w/2)
    center_y = y + int(h/2)
    cropped = transform.crop(img, (center_x,center_y), w, h)
    cv2.imwrite('/home/damian/test/gb'+str(i)+'.png', cropped)

In [52]:
os.getcwd()

'/home/damian/artia/painting-lab'